### Import packages

In [ ]:
library(GWmodel)      ### GW models
library(dplyr)
library(sp)           ## Data management
library(car)          ## vif
library(spdep)        ## Spatial autocorrelation
library(RColorBrewer) ## Visualization
library(classInt)     ## Class intervals
library(raster)       ## spatial data
library(grid)         # plot
library(gridExtra)    # Multiple plot
library(ggplot2)      # Multiple plot
library(gtable)
library(GGally)       # 相關係數圖矩陣（scatter plot matrix）
library(maptools)
library(MASS)
library(tmap)

### Function

#### pdf_plot

In [ ]:

pdf_plot <- function(x){
  g = ggplot()+
    geom_histogram(aes(x = x, y = ..density..), 
                   fill = '#557C55', alpha = 0.8)+
    geom_density(aes(x = x, y = ..density..), 
                 color = '#062C30', size = 1)+
    theme_bw()
  
  return(g)
}


### Load datas

In [63]:
getwd()
path = '..\\..\\Roaming-Dogs-Data\\'
Variable_KS_df <- read.csv(paste0(path, "@Test_KS\\Variable.csv" ), fileEncoding = 'utf-8')
Variable_KS_df["Clinic"][is.na(Variable_KS_df["Clinic"])] = 0
Variable_KS_shp<-shapefile(paste0(path, "@Test_KS\\Variable.shp" ))
# Variable_KS_centroid <-  st_centroid(Variable_KS_shp)
# colnames(Variable_KS_centroid)

ncol(Variable_KS_shp); nrow(Variable_KS_shp); colnames(Variable_KS_shp)

[1] "c:/@@Roaming_Dogs_Data_Analysis/Roaming-Dogs-Code/Model-fitting"

[1] 58

[1] 58

NULL

### Create a residual dataframe

In [ ]:
residual  <-  
data.frame(VILLCODE = Variable_KS_df$VILLCODE,
VILLNAME = Variable_KS_df$VILLNAME,
geometry = Variable_KS_centroid$geometry )

### select Xy

In [ ]:
col_X = c( "Market","Cluster","high_rat",   "mid_rat","low_rat","Hospital" ,  "Clinic", "M_F_RAT" ,
           "P_DEN",      "DEPENDENCY", "YOUN_DEP",   "OLD_DEP",    "AGING_IDX","Ele" ,"Ele_stu","Junior" ,   
           "Junior_stu", "Senior",     "Senior_stu", "TaxPayer" ,  "Income_tot", "Income_mea",
           "Income_med" ,"Income_Q1",  "Income_Q3",  "Income_sta", "Income_CV",  "Temple",    
           "Train.stat", "Train.crow" )
col_y = c('Nt')

In [ ]:
# crs(Variable_KS_shp)  <-  CRS('+init=EPSG:4326')


#### Correlation 

In [17]:
corr = cor(Variable_KS_df[c(col_y,col_X)])
# col_income = c( "Income_mea","Income_med" ,"Income_Q1", 
#                 "Income_Q3",  "Income_sta", "Income_CV")
# corr_income = sort(corr[col_income,'Nt'])
idx = abs(corr[,'Nt'])>.15
col_X_02 = names(corr[idx,'Nt']) %>% tail(-1)

In [18]:
col_X_02

[1] "Cluster"    "high_rat"   "low_rat"    "M_F_RAT"    "P_DEN"     
 [6] "DEPENDENCY" "YOUN_DEP"   "OLD_DEP"    "Ele"        "Income_mea"
[11] "Income_med" "Income_Q1"  "Income_Q3"  "Income_sta" "Income_CV" 
[16] "Temple"

### GLM_POISSON 

In [ ]:
Fit_Po <-glm(Nt~.,data=Variable_KS_df[,c(col_y,col_X_02)],family=poisson())
# Fit_Po <- glm(Nt~., data=Variable_KS_df[,c(col_y,col_X_02)], family = Gamma(link = "log"))
summary(Fit_Po) #查看回归模型参数


In [ ]:
# pdf_plot(Fit_Po$residuals)+xlab('residuals')
Variable_KS_centroid %>% 
  ggplot()+geom_sf(aes(color = residuals(Fit_Po), size = residuals(Fit_Po), alpha = .8))+
  scale_fill_gradient(low = "#56B1F7", high = "#132B43", na.value = NA)+
  theme_bw()

In [ ]:
nb <- poly2nb(Variable_KS_shp, queen=TRUE)
lw <- nb2listw(nb, style="W", zero.policy=TRUE)

In [ ]:
print(lw$weights[[1]])

### Spatial glmm
- [Package](https://kimura.univ-montp2.fr/~rousset/spaMM/spaMMintro.pdf)
- [Theory](https://bookdown.org/xiangyun/Thesis-Template-Bookdown/)

### GWR-Poisson

In [49]:
formula.GWPR = Nt ~.
DM<-gw.dist(dp.locat=data.matrix(((Variable_KS_df[c('X', "Y")]))))

In [65]:
bw.gwr <- bw.ggwr(formula.GWPR,  
                 data = (Variable_KS_shp[c(col_X_02, col_y)]),
                 family = "poisson",
                 approach = "AICc",
                 kernel = "gaussian", 
                 adaptive = TRUE,
                 dMat = DM )
bw.gwr

ERROR: Error in y + 0.1: 二元運算子中有非數值引數


In [109]:
df  <- lapply(Variable_KS_shp["Nt"],as.numeric)

ERROR: Error in as.list.default(X): 沒有方法可將這個 S4 類別強制變更成向量


3,8,11,3,2,19,5,3,1,3,⋯,31,26,11,9,2,1,16,2,1,2


In [100]:
for (i  in c(col_X_02, col_y)) {

    for (x in c(1:nrow(Variable_KS_shp))){

        Variable_KS_shp[x,i]  <-  as.numeric(Variable_KS_shp[x,i])
    }
   }
}

ERROR: Error in parse(text = x, srcfile = src): <text>:8:1: 未預期的 '}'
7:    }
8: }
   ^


In [101]:
print(Variable_KS_shp[58,"Nt"])

class       : SpatialPolygonsDataFrame 
features    : 1 
extent      : 120.3375, 120.3412, 22.60602, 22.61078  (xmin, xmax, ymin, ymax)
crs         : +proj=longlat +datum=WGS84 +no_defs 
variables   : 1
names       : Nt 
value       :  2 
